# Simulation of naive betting strategies

This notebook will demonstrate how to create simple simulations of Premier League seasons using different betting strategies.

In [1]:
import pandas as pd
import numpy as np
import datetime 

### Import and clean data
We will use 9 PL seasons in this example

In [2]:
training_seasons = ["pl1819","pl1718","pl1617","pl1516","pl1415","pl1314","pl1213","pl1112","pl1011"]
training_seasons.reverse()

Set the match dates as index. This needs to be done in order to simulate seasons in correct date order

In [3]:
df_list = []
for season in training_seasons:
    tmp = pd.read_csv("data/odds/pl/" + season + ".csv")
    tmp.dropna(axis=0, how="all", inplace=True)
    tmp.dropna(axis=1, how="all", inplace=True)
    if len(tmp[:1].Date.values[0].split("/")[2]) > 2: #2018
        tmp.index = pd.to_datetime(tmp.Date,format="%d/%m/%Y")
    else: #18
        tmp.index = pd.to_datetime(tmp.Date,format="%d/%m/%y")
        
    df_list.append(tmp)
df = pd.concat(df_list,sort=False)

In [4]:
display(df.head())
df.shape

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSH,PSD,PSA,PSCH,PSCD,PSCA
Date,,,,,,,,,,,,,,,,,,,,,
2010-08-14,E0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,...,1.45,1.40,3.00,2.78,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-14,E0,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,...,2.20,2.05,1.85,1.75,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-14,E0,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,...,1.60,1.55,2.56,2.36,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-14,E0,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,...,2.06,2.01,1.91,1.85,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-14,E0,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,...,1.64,1.55,2.50,2.38,NaN,NaN,NaN,NaN,NaN,NaN


(3420, 77)

In [5]:
def filter_team(df,name):
    return df[(df.HomeTeam == name) | (df.AwayTeam == name)].sort_index()

def home_games(df,name):
    return df[df.HomeTeam == name]

def away_games(df,name):
    return df[df.AwayTeam == name]

#Season format = xx/xx
def filter_season(df,season):
    return df.loc['20'+season.split("/")[0] + '-07-15':'20' + season.split("/")[1] + '-07-12'].sort_index()

def lastx_games(df,x):
     return filter_team(df[:5],row["HomeTeam"])[-x:] 
    
def get_teams(df):
    #array of teams this season
    return df.HomeTeam.unique()

In [6]:
filter_season(df,"15/16").head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSH,PSD,PSA,PSCH,PSCD,PSCA
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-08,E0,08/08/15,Bournemouth,Aston Villa,0.0,1.0,A,0.0,0.0,D,...,1.98,1.93,1.99,1.92,1.95,3.65,4.27,1.82,3.88,4.70
2015-08-08,E0,08/08/15,Chelsea,Swansea,2.0,2.0,D,2.0,1.0,H,...,2.24,2.16,1.80,1.73,1.39,4.92,10.39,1.37,5.04,10.88
2015-08-08,E0,08/08/15,Everton,Watford,2.0,2.0,D,0.0,1.0,A,...,2.28,2.18,1.76,1.71,1.70,3.95,5.62,1.75,3.76,5.44
2015-08-08,E0,08/08/15,Leicester,Sunderland,4.0,2.0,H,3.0,0.0,H,...,2.00,1.95,1.96,1.90,1.99,3.48,4.34,1.79,3.74,5.10
2015-08-08,E0,08/08/15,Man United,Tottenham,1.0,0.0,H,1.0,0.0,H,...,2.20,2.09,1.82,1.78,1.65,4.09,5.90,1.64,4.07,6.04


Define some useful columns

In [7]:
match_odds_cols = ["B365H","B365D","B365A"]
goals_odds_cols = ["BbAv>2.5", "BbAv<2.5"]
"""
FTHG and HG = Full Time Home Team Goals
FTAG and AG = Full Time Away Team Goals
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
"""
match_stat_cols = ["HST","AST","HS","AS","FTHG","FTAG"]
seasons=["10/11","11/12","12/13","13/14","14/15","15/16","16/17","17/18","18/19"]

In [8]:
df[match_odds_cols].idxmin(axis=1).head() #favourites in each game

Date
2010-08-14    B365H
2010-08-14    B365A
2010-08-14    B365H
2010-08-14    B365H
2010-08-14    B365H
dtype: object

### Simulate the seasons
simulate_seasons will iterate over all the seasons and place bets based on the favourite (min) or underdrog (max). The simulation will only bet on one match at a time, and refill the balance by 100 each time it loose. This is just a dummy example. A more realistic approach would be to group bets during the same day.

In [9]:
outcomes = ["H","D","A"]


def simulate_seasons(strategy):
    for season in seasons:
        seasonal_statistics = {}
        seasonal_bets = {}
        df_season = filter_season(df,season)
        teams = get_teams(df_season)

        balance = 100
        money_spent = 100
        max_balance = 0

        for team in teams:
            seasonal_statistics[team] = {
                "s": 0, #shots
                "ts": 0, #target shots
                "w": 0, #wins
                "d": 0, #draws
                "l": 0 #loss
            }

            seasonal_bets = {
                "correct": 0,
                "wrong": 0
            }

        print("Season ",season, " with ",strategy, " strategy")
        
        for _,match in df_season.iterrows():
            #.idxmax(axis=1)
            if strategy == "max":
                guess = outcomes[np.argmax(match[match_odds_cols].values)]
                value = np.max(match[match_odds_cols].values)
            if strategy == "min":
                guess = outcomes[np.argmin(match[match_odds_cols].values)]
                value = np.min(match[match_odds_cols].values)
                
            if guess == match.FTR: #Odds went in
                seasonal_bets["correct"] = seasonal_bets["correct"] + 1
                balance = balance * value
                if balance > max_balance:
                    max_balance = balance
            else:
                seasonal_bets["wrong"] = seasonal_bets["wrong"] + 1
                balance = 100 #Set inn 100 when you loose
                money_spent += 100 

        seasonal_bets["money_spent"] = money_spent
        seasonal_bets["max_balance"] = int(max_balance)

        print(seasonal_bets)
        
    
simulate_seasons("min")
print("\n\n")
simulate_seasons("max")

Season  10/11  with  min  strategy
{'correct': 193, 'wrong': 187, 'money_spent': 18800, 'max_balance': 12204}
Season  11/12  with  min  strategy
{'correct': 198, 'wrong': 182, 'money_spent': 18300, 'max_balance': 13150}
Season  12/13  with  min  strategy
{'correct': 202, 'wrong': 178, 'money_spent': 17900, 'max_balance': 7147}
Season  13/14  with  min  strategy
{'correct': 228, 'wrong': 152, 'money_spent': 15300, 'max_balance': 21066}
Season  14/15  with  min  strategy
{'correct': 204, 'wrong': 176, 'money_spent': 17700, 'max_balance': 15794}
Season  15/16  with  min  strategy
{'correct': 180, 'wrong': 200, 'money_spent': 20100, 'max_balance': 4336}
Season  16/17  with  min  strategy
{'correct': 232, 'wrong': 148, 'money_spent': 14900, 'max_balance': 74915}
Season  17/18  with  min  strategy
{'correct': 210, 'wrong': 170, 'money_spent': 17100, 'max_balance': 15903}
Season  18/19  with  min  strategy
{'correct': 223, 'wrong': 157, 'money_spent': 15800, 'max_balance': 130948}



Season  